本文中，我们将探讨混合线性注意力（Mixed Linear Attention，MLA）和 FlashAttention 这两个与 Transformer 相关的主题。

需要注意的是，“混合线性注意力”（MLA）这个术语不像 MHA, MQA, GQA 那样拥有一个单一、被广泛认可的标准定义。

它通常指的是结合了线性计算特性的注意力机制，可能包含多种变体。

我们将主要介绍与标准 Softmax 注意力形成对比的**线性注意力（Linear Attention）**的基本原理，并解释“混合”的可能性。

而 FlashAttention 是对标准 Softmax 注意力的一种**高效计算算法**，而非注意力机制本身的数学变化。

我们将分别进行解释、原理阐述、算法描述和基础的 Python (NumPy) 实现与测试。

---

### 1. 线性注意力（Linear Attention）

**注意：** "混合线性注意力"（MLA）不是一个标准术语，这里主要介绍**线性注意力（Linear Attention）**的概念，并阐述它与标准注意力的区别以及如何实现。

* **解释 (Explanation):**
    标准的缩放点积注意力（Scaled Dot-Product Attention）计算涉及到 $Q$ 和 $K$ 的点积 $QK^T$，生成一个形状为 $(L, L')$ 的注意力分数矩阵（$L$ 是 Query 序列长度，$L'$ 是 Key/Value 序列长度）。当序列很长时，这个矩阵会非常大 ($O(L \cdot L')$)，导致计算量 ($O(L \cdot L' \cdot d_{model})$) 和内存占用 ($O(L \cdot L')$) 都呈平方级别增长，成为 Transformer 处理长序列的瓶颈。

    线性注意力的目标是修改注意力计算方式，使其计算复杂度和内存占用相对于序列长度 $L$ 变成**线性**关系 ($O(L \cdot d_{model}^2)$ 或 $O(L \cdot d_{model} \cdot d_k)$，取决于实现)。它通常通过应用核技巧（Kernel Trick）来近似 Softmax 函数，从而改变计算顺序，避免计算并存储完整的 $L \times L'$ 注意力矩阵。

    “混合线性注意力”可能指的是在一个模型中同时使用标准的 Softmax 注意力（可能用于短距离或关键部分）和线性注意力（用于长距离），或者注意力机制的某些部分是线性的。

* **数学原理 (Mathematical Principle):**
    标准 Softmax 注意力计算为：
    $$\text{Attention}(Q, K, V) = \text{softmax}(\frac{QK^T}{\sqrt{d_k}})V$$
    展开来看，对于 Query 中的第 $i$ 个向量 $q_i$ 和 Value 中的所有向量 $V = [v_1, ..., v_{L'}]^T$，输出的第 $i$ 个向量是：
    $$\text{Output}_i = \sum_{j=1}^{L'} \frac{\exp(\frac{q_i \cdot k_j}{\sqrt{d_k}})}{\sum_{m=1}^{L'} \exp(\frac{q_i \cdot k_m}{\sqrt{d_k}})} v_j$$
    注意，分母 $\sum_{m=1}^{L'} \exp(\frac{q_i \cdot k_m}{\sqrt{d_k}})$ 依赖于 $q_i$，这使得计算必须先得到完整的 $QK^T$ 矩阵。

    线性注意力通过使用一个**核函数** $\phi(\cdot) > 0$ 来近似 $\exp(\cdot)$，并重写计算公式。如果 Softmax 可以近似为 $\text{softmax}(\frac{q_i \cdot k_j}{\sqrt{d_k}}) \approx \frac{\phi(q_i)^T \phi(k_j)}{\sum_{m=1}^{L'} \phi(q_i)^T \phi(k_m)}$，那么输出可以重写为：
    $$\text{Output}_i = \sum_{j=1}^{L'} \frac{\phi(q_i)^T \phi(k_j)}{\sum_{m=1}^{L'} \phi(q_i)^T \phi(k_m)} v_j$$
    $$\text{Output}_i = \frac{\phi(q_i)^T \sum_{j=1}^{L'} \phi(k_j) v_j}{\phi(q_i)^T \sum_{m=1}^{L'} \phi(k_m)}$$
    关键在于，项 $\sum_{j=1}^{L'} \phi(k_j) v_j$（形状 $(d_\phi, d_v)$，其中 $d_\phi$ 是 $\phi$ 输出的维度）和项 $\sum_{m=1}^{L'} \phi(k_m)$（形状 $(d_\phi,)$）**不依赖于 Query**。它们可以先计算出来，计算量是 $O(L' \cdot d_\phi \cdot d_v + L' \cdot d_\phi)$。然后，对于每个 $q_i$，只需要与这些预计算的结果进行点积和向量除法，计算量是 $O(d_\phi^2 + d_\phi \cdot d_v)$。总的计算量就变成了 $O(L' \cdot d_\phi \cdot d_v + L \cdot d_\phi \cdot d_v)$，即相对于序列长度 $L$ 和 $L'$ 是线性的，只要 $d_\phi$ 和 $d_v$ 远小于 $L, L'$。

    常用的核函数 $\phi(x)$ 包括 $\exp(\text{elu}(x) + 1)$，或者简单使用 $\exp(x)$（需要小心数值稳定性）。维度 $d_\phi$ 通常与 $d_k$ 或 $d_{model}$ 相关。

* **算法 (Algorithm Steps):**
    以 $\phi(x) = \exp(x/\sqrt{d_k})$ 为例，使用核函数应用于 Q 和 K：
    1.  输入 $Q, K, V$ (形状通常为 $(B, L, d_{model})$)。通过线性投影得到 $Q', K', V'$，形状 $(B, L, d_k), (B, L', d_k), (B, L', d_v)$。
    2.  对 $Q'$ 和 $K'$ 的元素应用核函数 $\phi(\cdot)$：得到 $\phi(Q')$ (形状 $(B, L, d_k)$) 和 $\phi(K')$ (形状 $(B, L', d_k)$)。
    3.  计算 Key-Value 积分项（或称作 Context 累积）：$\text{KV}_{sum} = \sum_{j=1}^{L'} \phi(K')_j^T V'_j$。在批次维度上并行，在序列长度 $L'$ 上求和。形状 $(B, d_k, d_v)$。
    4.  计算 Key 的归一化项：$\text{K}_{norm} = \sum_{j=1}^{L'} \phi(K')_j$。在批次维度上并行，在序列长度 $L'$ 上求和。形状 $(B, d_k)$。
    5.  计算输出：对于每个 $i$，$\text{Output}_i = \phi(Q')_i \cdot \text{KV}_{sum} / (\phi(Q')_i \cdot \text{K}_{norm})$。这里的乘法是向量点积或矩阵乘法，除法是元素级的。形状 $(B, L, d_v)$。

* **Python 代码实现与测试 (Python Code Implementation & Test):**
    我们将实现一个基本的线性注意力版本，使用 $\phi(x) = \exp(x)$（并进行一些数值稳定性处理）。为了简化，这里假设 $d_k = d_v = d_{model}$.

In [3]:
import numpy as np
import math

def softmax(x, axis=-1):
    """Compute softmax values for each sets of scores in x."""
    # For numerical stability
    x = x - np.max(x, axis=axis, keepdims=True)
    e_x = np.exp(x)
    return e_x / np.sum(e_x, axis=axis, keepdims=True)

def scaled_dot_product_attention(q, k, v, mask=None):
    """
    计算缩放点积注意力。

    Args:
        q: Query 张量，形状 (..., seq_len_q, depth)。
        k: Key 张量，形状 (..., seq_len_k, depth)。
        v: Value 张量，形状 (..., seq_len_v, depth_v)。
            seq_len_k 必须等于 seq_len_v。
        mask: Mask 张量 (用于屏蔽某些连接)，形状 (..., seq_len_q, seq_len_k)。
              通常在解码器中使用 (Look-ahead mask) 或处理变长序列 (Padding mask)。

    Returns:
        output: 注意力计算结果，形状 (..., seq_len_q, depth_v)。
        attention_weights: 注意力权重，形状 (..., seq_len_q, seq_len_k)。
    """
    matmul_qk = np.matmul(q, k.transpose(0, 1, 3, 2)) # (..., seq_len_q, seq_len_k)

    d_k = q.shape[-1]
    scaled_attention_logits = matmul_qk / np.sqrt(d_k)

    if mask is not None:
        # 将 mask 为 0 的位置 (即需要忽略的位置) 的 logits 设置为一个非常小的负数
        scaled_attention_logits = scaled_attention_logits + (mask * -1e9)

    attention_weights = softmax(scaled_attention_logits, axis=-1) # (..., seq_len_q, seq_len_k)
    output = np.matmul(attention_weights, v) # (..., seq_len_q, depth_v)

    return output, attention_weights

def split_heads(x, num_heads, depth):
    """
    将最后一个维度分割成 (num_heads, depth)。
    转置结果以便进行 attention 计算。

    Args:
        x: 输入张量，形状 (batch_size, seq_len, d_model)。
        num_heads: 头数。
        depth: 每个头的维度 (d_model / num_heads)。

    Returns:
        一个形状为 (batch_size, num_heads, seq_len, depth) 的张量。
    """
    x = x.reshape(x.shape[0], x.shape[1], num_heads, depth)
    return x.transpose(0, 2, 1, 3)

def combine_heads(x):
    """
    合并多头注意力的输出。

    Args:
        x: 输入张量，形状 (batch_size, num_heads, seq_len, depth)。

    Returns:
        一个形状为 (batch_size, seq_len, d_model) 的张量。
    """
    x = x.transpose(0, 2, 1, 3)
    d_model = x.shape[2] * x.shape[3]
    return x.reshape(x.shape[0], x.shape[1], d_model)

# Helper for initializing dummy weights
def init_weights(*shape):
    return np.random.randn(*shape).astype(np.float32) * 0.01 # Use small random values

# Helper for creating a padding mask (example)
def create_padding_mask(seqs, pad_token=0):
    """
    创建用于注意力机制的填充掩码。
    Args:
        seqs: 输入序列张量 (batch_size, seq_len)。
        pad_token: 填充 token 的 ID。
    Returns:
        一个形状为 (batch_size, 1, 1, seq_len_k) 的掩码张量，填充位置为 1，否则为 0。
    """
    mask = (seqs == pad_token)[:, np.newaxis, np.newaxis, :]
    return mask.astype(np.float32)

# Helper for creating a look-ahead mask (example for decoder self-attention)
def create_look_ahead_mask(seq_len):
    """
    创建用于解码器自注意力的前瞻掩码。
    Args:
        seq_len: 序列长度。
    Returns:
        一个形状为 (1, 1, seq_len, seq_len) 的掩码张量，需要屏蔽的位置为 1，否则为 0。
    """
    mask = 1 - np.tril(np.ones((seq_len, seq_len), dtype=np.float32))
    return mask[np.newaxis, np.newaxis, :, :].astype(np.float32)



import numpy as np
def linear_attention(q, k, v, mask=None):
    """
    实现一个基础的线性注意力变体 (使用 exp 核)。
    这个实现是为了展示线性注意力的计算顺序，可能不是最高效或数值最稳定的。
    Args:
        q: Query 张量，形状 (batch_size, seq_len_q, d_model)。
        k: Key 张量，形状 (batch_size, seq_len_k, d_model)。
        v: Value 张量，形状 (batch_size, seq_len_v, d_model)。
            seq_len_k 必须等于 seq_len_v。
        mask: 填充掩码，形状 (batch_size, 1, 1, seq_len_k)。注意：线性注意力中的掩码应用方式与 Softmax 注意力不同。
              这里为了简化，我们假设 mask 只需将 padding 位置的 phi(K) 设为 0。

    Returns:
        output: 线性注意力计算结果，形状 (batch_size, seq_len_q, d_model)。
        # 注意：线性注意力通常不返回注意力权重矩阵，因为它不被显式计算。
    """
    # 确保 d_k = d_v = d_model for this simplified example
    assert q.shape[-1] == k.shape[-1] == v.shape[-1]
    d_model = q.shape[-1]
    batch_size, seq_len_q, _ = q.shape
    _, seq_len_k, _ = k.shape # assuming seq_len_k == seq_len_v

    # 应用核函数 (这里简化使用 np.exp)
    # 为了数值稳定性，通常会减去最大值，但这会影响 exp 的正性，需要更复杂的核函数
    # 这里直接用 exp，注意实际应用中可能需要更好的核或数值技巧
    phi_q = np.exp(q) # (batch_size, seq_len_q, d_model)
    phi_k = np.exp(k) # (batch_size, seq_len_k, d_model)

    # 应用掩码到 phi_k (将 padding 位置的 K 的 phi 值设为接近 0)
    if mask is not None:
      # 掩码通常是 0 或 1，我们需要一个乘法掩码 (1 表示保留，0 表示屏蔽)
      # 如果输入 mask 是 padding=1 的那种，我们需要反转
      mul_mask = (1 - mask) # padding=1 -> mul_mask=0; non-padding=0 -> mul_mask=1
      print(f"mul_mask shape: {mul_mask.shape}")
      print(f"phi_k initial shape: {phi_k.shape}")
      # Corrected mask application:
      # phi_k has shape (batch_size, seq_len_k, d_model)
      # mul_mask has shape (batch_size, 1, 1, seq_len_k)
      # We need the effective mask to be (batch_size, seq_len_k, 1) for broadcasting.
      squeezed_mul_mask = mul_mask.squeeze(axis=(1, 2)) # Shape: (batch_size, seq_len_k)
      # print(f"squeezed_mul_mask for phi_k shape: {squeezed_mul_mask.shape}") # Optional debug print
      phi_k = phi_k * squeezed_mul_mask[:, :, np.newaxis] # Broadcasts (B, Lk, D) * (B, Lk, 1) -> (B, Lk, D)

    # 3. 计算 KV 积分项 (Context 累积)
    # 形状 (batch_size, d_model, d_model) - sum over seq_len_k dimension
    # (B, L', d) transpose to (B, d, L'), (B, L', d_v) -> (B, d, d_v)
    kv_sum = np.matmul(phi_k.transpose(0, 2, 1), v)

    # 4. 计算 K 的归一化项 (sum over seq_len_k dimension)
    # 形状 (batch_size, d_model)
    k_norm = np.sum(phi_k, axis=1) # sum over seq_len_k dimension

    # 为了数值稳定性，避免除以零，给 k_norm 加一个小的 epsilon
    k_norm = k_norm + 1e-6

    # 5. 计算输出
    # (B, L, d) matmul (B, d, d_v) -> (B, L, d_v)
    numerator = np.matmul(phi_q, kv_sum)

    # Normalize numerator by k_norm
    # (B, L, d_v) / (B, d) - This requires careful broadcasting or expansion
    # k_norm is (B, d_model). We need to divide each row of numerator[b, i, :] by k_norm[b, :]
    # Expand k_norm to (B, 1, d_model) for broadcasting
    denominator = k_norm[:, np.newaxis, :] # (batch_size, 1, d_model)

    output = numerator / denominator # Element-wise division due to broadcasting

    return output # No attention weights returned


In [4]:

# --- Test Linear Attention ---
print("--- Test Linear Attention ---")
batch_size = 2
seq_len_q = 4
seq_len_k = 5
d_model = 64 # Assuming d_k = d_v = d_model for simplicity

# Simulate input tensors
q_lin = init_weights(batch_size, seq_len_q, d_model)
k_lin = init_weights(batch_size, seq_len_k, d_model)
v_lin = init_weights(batch_size, seq_len_k, d_model)

# Simulate a padding mask (same as MHA)
dummy_keys_for_mask_lin = np.array([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0]]) # 1s are actual tokens, 0s are padding
lin_mask = create_padding_mask(dummy_keys_for_mask_lin, pad_token=0) # Shape (2, 1, 1, 5)
print(f"Linear Attention Mask shape: {lin_mask.shape}")


lin_output = linear_attention(q_lin, k_lin, v_lin, mask=lin_mask)

print(f"Linear Attention Input Q shape: {q_lin.shape}")
print(f"Linear Attention Input K shape: {k_lin.shape}")
print(f"Linear Attention Input V shape: {v_lin.shape}")
print(f"Linear Attention Output shape: {lin_output.shape}") # Expected: (2, 4, 64)
print("-" * 30)

--- Test Linear Attention ---
Linear Attention Mask shape: (2, 1, 1, 5)
mul_mask shape: (2, 1, 1, 5)
phi_k initial shape: (2, 5, 64)
Linear Attention Input Q shape: (2, 4, 64)
Linear Attention Input K shape: (2, 5, 64)
Linear Attention Input V shape: (2, 5, 64)
Linear Attention Output shape: (2, 4, 64)
------------------------------


**线性注意力计算与优化:**

上面的基础实现展示了核心的计算顺序变化。实际的线性注意力实现会有很多优化和变体：

* **核函数的选择：** 不同的核函数影响模型的表达能力和数值稳定性。
* **数值稳定性：** `exp` 函数容易溢出或下溢。实际实现会使用各种技巧（如 LogSumExp 或其他数值稳定的核）。
* **分组或多头：** 线性注意力也可以结合多头机制，通常是对 $d_{model}$ 维度进行分割。
* **Query/Key 归一化：** 有些线性注意力变体会对 Q 和 K 进行 L2 归一化。
* **实现效率：** 在硬件上高效实现矩阵乘法和归约操作。

---

### 2. FlashAttention

* **全文 (Full Name):** FlashAttention
* **解释 (Explanation):** FlashAttention 是由 Tri Dao 等人提出的，它是**标准缩放点积注意力**的一种**硬件感知的高效计算方法**，而不是改变了注意力机制的数学定义。它的核心创新在于利用 GPU 的 SRAM（高速缓存，容量小但速度快）来存储中间计算结果，避免将大的 $L \times L$ 注意力矩阵 (QK^T 和 Softmax 后的 P) 写回低速但大容量的 HBM（显存）。通过巧妙的平铺（Tiling）策略和“在线 Softmax”（Online Softmax）计算，FlashAttention 显著减少了 HBM 的读写次数，从而加速计算并降低显存使用。
* **数学原理 (Mathematical Principle):**
    FlashAttention 计算的**数学函数**与标准注意力完全相同：
    $$\text{Attention}(Q, K, V) = \text{softmax}(\frac{QK^T}{\sqrt{d_k}})V$$
    核心的数学技巧在于如何**数值稳定地**计算 Softmax 并累积 $P \cdot V$ 的结果，而无需存储完整的 $P$ 矩阵。利用 Softmax 的性质：
    $$ \text{softmax}(x_i) = \frac{e^{x_i}}{\sum_j e^{x_j}} = \frac{e^{x_i - M}}{\sum_j e^{x_j - M}}$$
    其中 $M$ 可以是任何常数。FlashAttention 将 Softmax 计算的输入向量 $x$（即 $q_i \cdot k_j / \sqrt{d_k}$ 构成的行向量）分割成多个块，并迭代地处理这些块。在处理每个块时，它会更新一个运行的最大值 $M_{new}$ 和一个运行的归一化常数 $\sum e^{x_j - M_{new}}$。

    假设 Softmax 的输入向量 $x$ 分成两块 $[x_{1:B}, x_{B+1:L'}]$. 标准计算是 $P = \text{softmax}([x_{1:B}, x_{B+1:L'}])$。在线计算可以这样做：
    1. 计算第一块的 Softmax：$P_{1:B} = \text{softmax}(x_{1:B})$。得到归一化常数 $Z_1$ 和最大值 $M_1 = \max(x_{1:B})$。计算部分输出 $O_{1:B} = P_{1:B} V_{1:B}$。
    2. 计算第二块的 Softmax：$P_{B+1:L'} = \text{softmax}(x_{B+1:L'})$。得到归一化常数 $Z_2$ 和最大值 $M_2 = \max(x_{B+1:L'})$。计算部分输出 $O_{B+1:L'} = P_{B+1:L'} V_{B+1:L'}$。
    3. 合并两个 Softmax 并更新归一化和输出：
        新的最大值 $M = \max(M_1, M_2)$。
        新的归一化常数 $Z = e^{M_1 - M} Z_1 + e^{M_2 - M} Z_2$.
        最终的 Softmax 是 $\text{softmax}(x) = \frac{e^{x - M}}{Z}$。
        最终的输出是 $O = \frac{e^{M_1 - M} Z_1}{Z} O_{1:B} + \frac{e^{M_2 - M} Z_2}{Z} O_{B+1:L'}$。

    这个过程可以推广到任意数量的块，迭代地更新最大值 $M$ 和归一化常数 $Z$，并累积加权的输出块。

* **算法 (Algorithm Steps):**
    FlashAttention 算法的核心在于如何高效地将 Q, K, V 分割成块，并在 GPU 的 SRAM 中执行计算：
    1.  将 Q 矩阵按行分块，V 矩阵按行分块，K 矩阵按列分块。每次加载 Q 的一个块 $Q_i$ 和 K、V 的一个块 $K_j, V_j$ 到 SRAM。
    2.  在 SRAM 中计算 $S_{ij} = Q_i K_j^T / \sqrt{d_k}$。
    3.  使用在线 Softmax 算法，结合当前块 $S_{ij}$ 和之前块累积的最大值和归一化常数，计算当前块的注意力权重 $P_{ij}$。同时更新累积的最大值和归一化常数。
    4.  计算当前输出块的贡献 $P_{ij} V_j$ 并累加到总的输出块 $O_i$ 中。
    5.  对 K, V 的所有块 $j$ 重复步骤 1-4，处理完一整行 $Q_i$ 的所有 Key/Value 对。
    6.  将计算好的输出块 $O_i$ 写回 HBM。
    7.  对 Q 的所有块 $i$ 重复步骤 1-6。

    这个算法的关键在于整个过程中的 Softmax 归一化和输出累积都在 SRAM 中完成，避免了 $O(L \times L')$ 大小的 $S$ 和 $P$ 矩阵写入 HBM。

* **Python 代码实现与测试 (Python Code Implementation & Test):**
    如前所述，用 NumPy 实现 FlashAttention 的核心性能优势（降低 HBM 访问）是不可能的，因为 NumPy 操作在内存管理上没有细粒度控制，并且不直接暴露 GPU SRAM。我们在这里提供一个简化的 NumPy 示例，仅用于**演示在线 Softmax 的数学逻辑**，展示如何分块计算 Softmax 并合并结果，而不是一个高性能的 FlashAttention 实现。

In [5]:
# --- Basic NumPy Demo of Online Softmax Logic (Not FlashAttention Performance) ---
def online_softmax_demo(scores_row, block_size):
    """
    演示在线 Softmax 的数学逻辑。
    计算一个向量 (代表 QK^T 的一行分数) 的 Softmax，分块处理。
    Args:
        scores_row: QK^T 的一行分数，形状 (seq_len_k,)。
        block_size: 每次处理的块大小。
    Returns:
        该行分数的 Softmax 结果。
    """
    seq_len_k = scores_row.shape[0]
    # 初始化运行的最大值和归一化常数
    m_prev = -float('inf') # Previous maximum, initialized to negative infinity
    l_prev = 0.0            # Previous normalizer sum, initialized to 0

    output_row = np.zeros_like(scores_row) # To store the resulting softmax probabilities

    for i in range(0, seq_len_k, block_size):
        block_end = min(i + block_size, seq_len_k)
        scores_block = scores_row[i:block_end]

        # 计算当前块的最大值
        m_curr = np.max(scores_block)

        # 计算新的全局最大值
        m_new = np.maximum(m_prev, m_curr)

        # 更新归一化常数
        # L_new = exp(m_prev - m_new) * L_prev + exp(m_curr - m_new) * sum(exp(scores_block - m_curr))
        l_new = np.exp(m_prev - m_new) * l_prev + np.sum(np.exp(scores_block - m_curr))

        # 计算当前块的 Softmax 概率 (使用新的全局最大值和归一化常数)
        # P_block = exp(scores_block - m_new) / L_new
        softmax_block = np.exp(scores_block - m_new) / l_new

        # 更新运行变量给下一个块
        m_prev = m_new
        l_prev = l_new

        # 将计算好的 Softmax 块放入输出
        output_row[i:block_end] = softmax_block

    return output_row

# --- Test Online Softmax Demo ---
print("--- Test Online Softmax Demo ---")
seq_len = 10
block_size = 3
# Simulate a row of scores (e.g., from QK^T / sqrt(d_k))
dummy_scores = np.random.rand(seq_len) * 10 # Using larger values to show potential overflow/underflow mitigation

print(f"Original scores row: {dummy_scores}")

# Calculate standard softmax for comparison
standard_softmax = softmax(dummy_scores, axis=-1)

# Calculate online softmax
online_sm_result = online_softmax_demo(dummy_scores, block_size)

print(f"Standard Softmax result: {standard_softmax}")
print(f"Online Softmax demo result: {online_sm_result}")

# Check if results are close
print(f"Results are close: {np.allclose(standard_softmax, online_sm_result)}")
print("-" * 30)

--- Test Online Softmax Demo ---
Original scores row: [0.59885727 4.17296555 9.17626131 8.75696076 1.12491781 4.37783372
 1.52455393 2.9992116  8.44856526 2.23537473]
Standard Softmax result: [8.72017462e-05 3.10985892e-03 4.63067631e-01 3.04470003e-01
 1.47567382e-04 3.81692652e-03 2.20064573e-04 9.61580882e-04
 2.23671189e-01 4.47976964e-04]
Online Softmax demo result: [1.87021981e-04 6.66972854e-03 9.93143249e-01 3.25510372e-01
 1.57765010e-04 4.08069484e-03 1.57090697e-04 6.86414033e-04
 1.59665241e-01 2.40338269e-04]
Results are close: False
------------------------------


**FlashAttention 性能分析:**

* **计算复杂度:** FlashAttention 的**浮点运算次数 (FLOPs)** 与标准注意力是相同的，仍然是 $O(L^2 \cdot d_{model})$（在 Q 和 K/V 序列长度相近时）。
* **内存复杂度:** FlashAttention 的**中间显存占用**（主要是 $QK^T$ 和 $P$ 矩阵）从 $O(L^2)$ 降到 $O(\sqrt{L} \cdot d_{model})$ 或者 $O(d_{model}^2)$（取决于平铺块大小）。K/V 缓存的内存占用如果是 MHA 仍然是 $O(L \cdot d_{model} \cdot h)$，如果是 MQA/GQA 则更低，FlashAttention 不改变 K/V 缓存结构本身，但由于能处理更长的序列，整体显存压力降低。
* **性能瓶颈:** 标准注意力的性能瓶颈通常在于 HBM 的**带宽**（读写速度），因为需要频繁读写大的中间矩阵。FlashAttention 通过在 SRAM 中完成大部分计算，将瓶颈转移到**计算密度**（FLOPs 的执行速度）。
* **实际速度提升:** 根据序列长度和硬件，FlashAttention 可以比标准实现快 2-4 倍，对于更长的序列提升更显著。
* **显存节省:** 可以节省 20-50% 的显存，从而允许训练或推理时使用更长的序列，或更大的模型批次大小。
* **适用场景:** 尤其适用于 GPU 等具有 SRAM 和 HBM 分级存储的硬件。在 CPU 上实现意义不大，因为 CPU 缓存结构不同且通常没有 HBM 带宽瓶颈。

总的来说，线性注意力旨在通过改变数学公式来降低计算复杂度，从而从根本上解决长序列的 $O(L^2)$ 问题（以牺牲近似 Softmax 带来的潜在性能损失为代价）。而 FlashAttention 则是在不改变数学公式的前提下，通过优化底层计算和内存访问模式，在现有硬件上更高效地执行标准的 $O(L^2)$ 注意力计算。